# Final exam programming problem

In this problem, you will work on the following:

- For a graph G, you will compute its MST
- We will add an edge into the MST and you should be able to quickly update the MST

To solve this problem, you may consult three lab sessions:

- Merge sort (We will sort the edges using merge sort)
- MST (Kruskal's algorithm)
- DFS (To find a cycle on a graph)

In [1]:
import random

## Part 1

You must work with the provided data structures.
You may use your own data structures if they are not defined here.

In [2]:
# Define graph edge
class Edge:
    def __init__(self, node1, node2, weight=1.0):
        self.node1 = node1
        self.node2 = node2
        self.weight = weight
        
    def __lt__(self, other):
        selfPriority = self.weight
        otherPriority = other.weight
        return selfPriority < otherPriority

In [3]:
# Define an undirected graph
class UndirectedGraph:
    def __init__(self, n):
        self.num_nodes = n
        self.nodes = [set() for i in range(n)]
        self.edges = []
    
    # edge node1 <--> node2 (undirected)
    def insert(self, node1, node2, weight=1.0):
        self.nodes[node1].add(node2)
        self.nodes[node2].add(node1)
        self.edges.append(Edge(node1, node2, weight))
        self.edges.append(Edge(node2, node1, weight))

### Graph representation

- Linked list: self.nodes in class UndirectedGraph (for DFS)
- Edge list: self.edges in class UndirectedGraph (for MST)

In [4]:
# A function to generate a random graph
def generate_graph_and_output_edges(num_nodes, random_seed, num_edge_per_node=3):
    random.seed(random_seed)
    graph = UndirectedGraph(num_nodes)
    edge_list = []
    for i in range(num_nodes):
        while len(graph.nodes[i]) < num_edge_per_node:
            node = random.randint(0, num_nodes-1)
            if node != i and (node not in graph.nodes[i]):
                weight = random.random()
                graph.insert(i, node, weight)
    return graph  

### A small sample graph

This is a sample graph so you can learn how graph is represented in this problem

In [5]:
# Generate the graph
graph_sample = generate_graph_and_output_edges(5, 100, 2)

# print nodes represented in linked list
print(graph_sample.nodes)

# print edges represented in edge list
for e in graph_sample.edges:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

[{1, 2, 3}, {0, 3, 4}, {0, 3, 4}, {0, 1, 2}, {1, 2}]
 0,  1, 0.45953
 1,  0, 0.45953
 0,  3, 0.73196
 3,  0, 0.73196
 1,  3, 0.50687
 3,  1, 0.50687
 2,  0, 0.53290
 0,  2, 0.53290
 2,  3, 0.26342
 3,  2, 0.26342
 4,  1, 0.33535
 1,  4, 0.33535
 4,  2, 0.83890
 2,  4, 0.83890


In [6]:
# Union Find data structure is provided to support Kruskal
class UnionFind:
    def __init__(self, num_nodes):
        # Initially position[i] = i
        self.position = [i for i in range(num_nodes)]
        
    # Return the cluster index
    def find(self, node):
        if self.position[node] == node:
            return node
        else:
            self.position[node] = self.find(self.position[node])
            return self.position[node]
    
    def union(self, node1, node2):
        a = self.find(node1)
        b = self.find(node2)
        # no need to union
        if a == b:
            return
        # union is needed
        else:
            if a < b:
                self.position[b] = a
            else:
                self.position[a] = b

## Part 2

Find a MST for a given graph.

The returned MST should be in edge list format. (Just like what we did in the lab)

When sorting the graph.edges, you must use Merge Sort. (We used PriorityQueue in the lab)

In [9]:
# Implement MergeSort

# Your merge sort should returend a sorted list of A
def merge(X, Y):
    m = len(X)
    n = len(Y)  

    # output list to store the merged (and sorted) X and Y.
    output = []

    
    i = 0 #index for X
    j = 0 #index for Y

    #length of the output list should be the sum of the length of the two lists to be merged
    while len(output) < (m+n):
        #get the smaller element of X and Y at the current i and j and add to the output.
        if X[i] < Y[j]:
            output.append(X[i])
            i += 1
            #if we have run through the entire X, append the remaining Y (the remaining will already be sorted)
            if i>=len(X):
                output.extend(Y[j:])
        else:
            output.append(Y[j])
            j += 1
            #if we have run through the entire Y, append the remaining X (the remaining will already be sorted)
            if j>=len(Y):
                output.extend(X[i:])

    return output


def MergeSort(A):
    n = len(A)

    #base case
    if n<=1:
        return A

    else: 
        mid = n//2
        left_half = A[:mid]
        right_half = A[mid:]
        left_half = MergeSort(left_half) #sorted left half
        right_half = MergeSort(right_half) #sorted right half
        return merge(left_half,right_half) #combine(merge) the two sorted halves

In [22]:
graph_sample_edge_weights  = [] 
for e in graph_sample.edges:
    graph_sample_edge_weights.append(e.weight)

sorted_edge_weights = MergeSort(graph_sample_edge_weights)

In [62]:
# Implement Kruskal

# returned MST is a list of class Edge
def kruskal(graph):
    MST = []
    uf = UnionFind(graph.num_nodes)

    graph_edge_weights  = [] 
    for e in graph.edges:
        graph_edge_weights.append(e.weight)

    sorted_edge_list = MergeSort(graph_edge_weights)

    counter = 0 
    while counter < len(sorted_edge_list):
        edge_weight = sorted_edge_list[counter]
        src = [e.node1 for e in graph.edges if e.weight == edge_weight][0]
        dst = [e.node2 for e in graph.edges if e.weight == edge_weight][0]
        counter += 1
        if uf.find(src) != uf.find(dst):
            MST.append((Edge(src, dst, edge_weight)))
            uf.union(src,dst)

    return MST

## Part 3

A new edge (2, 19) with weight 0.2

We need to quickly update the old MST to get a new MST

In [73]:
# Construct a graph from edge list. This is given to you.

# We assume num_nodes is given for simplicity.
def generate_graph_from_edges(edge_list, num_nodes):
    graph = UndirectedGraph(num_nodes)
    for e in edge_list:
        graph.insert(e.node1, e.node2, e.weight)
    return graph

### Algorithm

- We add the new edge to the old MST which will generate a cycle

- The cycle consists at least three edges

- The edge with the largest weight should be removed

- You ONLY need to print out the edges on the cycle to get full marks (the two end nodes and the weight)

### Sample example

- We have a MST for a graph of 3 nodes

- MST has edge (0, 1, 0.12) and (0, 2, 0.23)

- We add a new edge (1, 2, 0.2)

- A cycle of three edges is generated

- We need to delete edge (0, 2, 0.23)

### How to use DFS to find the cycle

- Suppose the new edge is (2, 19, 0.2)

- If we DFS from node 2, we will eventurally hit node 19

- You need to properly append and pop on "cycle_edges" during DFS 

- The "cycle_edge" should contain the final path from 2 to 19

In [80]:
# Use DFS to detect the cycle

# You may add more arguments in dfs_recursive() for your convenience

def dfs(graph, starting_node, stopping_node):
    visited_nodes = [False for i in range(graph.num_nodes)]
    cycle_edges = []
    dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node)
    
    # The following is for demonstration purpose
    print("We found {:d} edges from node {:d} to node {:d}"
          .format(len(cycle_edges), starting_node, stopping_node))
    for e in cycle_edges:
        '''
        print the two end nodes of the edge
            the weight information is not known so you do not need to output 
        
        You may change the following print statement
            if you did not use Edge class
        '''
        print("{:2d}, {:2d}".format(e.node1, e.node2))


def dfs_recursive(graph, starting_node, visited_nodes, cycle_edges, stopping_node):

    if starting_node == stopping_node:
        return
    else:
        visited_nodes[starting_node] = True
        neighbors = graph.nodes[starting_node]
        if all([visited_nodes[neigh] for neigh in neighbors]):
            return

        for neigh in neighbors:    
            if visited_nodes[neigh]:
                continue
            visited_nodes[starting_node] = True
            cycle_edges.append((Edge(starting_node, neigh)))
            dfs_recursive(graph, neigh, visited_nodes,cycle_edges,stopping_node)

        return

    

## Part 4 Test code

Do NOT modify the code in this part.


In [63]:
graph_mst = generate_graph_and_output_edges(20, 10, 5)
MST = kruskal(graph_mst)

for e in MST:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

 3, 18, 0.00406
 5,  1, 0.03175
 0, 18, 0.03259
 0,  5, 0.03440
 1, 11, 0.04456
18,  2, 0.05057
13, 15, 0.06277
 4, 15, 0.06499
11,  7, 0.07993
19,  4, 0.10876
 9,  8, 0.14351
 5, 16, 0.15642
10,  9, 0.16494
 7, 14, 0.16636
 9, 16, 0.17846
 3,  4, 0.19495
 6,  4, 0.38442
12,  5, 0.39059
17,  0, 0.43858


In [74]:
def find_the_cycle(MST, new_edge, num_nodes):
    # construct graph from edge list
    graph_cycle = generate_graph_from_edges(MST, num_nodes)

    # call dfs to find a path from node1 to node2
    dfs(graph_cycle, new_edge.node1, new_edge.node2)

In [84]:
find_the_cycle(MST, Edge(2, 19, 0.2), 20)

We found 19 edges from node 2 to node 19
 2, 18
18,  0
 0, 17
 0,  5
 5,  1
 1, 11
11,  7
 7, 14
 5, 12
 5, 16
16,  9
 9,  8
 9, 10
18,  3
 3,  4
 4, 19
 4,  6
 4, 15
15, 13


In [85]:
# Instead we can add edge (3, 14, 0.1)
find_the_cycle(MST, Edge(3, 14, 0.1), 20)

We found 19 edges from node 3 to node 14
 3, 18
18,  0
 0, 17
 0,  5
 5,  1
 1, 11
11,  7
 7, 14
 5, 12
 5, 16
16,  9
 9,  8
 9, 10
18,  2
 3,  4
 4, 19
 4,  6
 4, 15
15, 13


### Hint

If you run the following codes,

In [81]:
graph_hint = generate_graph_and_output_edges(10, 1, 2)

mst = kruskal(graph_hint)

print("The minimum spanning tree:")
for e in mst:
    print("{:2d}, {:2d}, {:.5f}".format(e.node1, e.node2, e.weight))

find_the_cycle(mst, Edge(1, 8, 0.2), 10)

The minimum spanning tree:
 6,  8, 0.00920
 5,  0, 0.02232
 0,  1, 0.25507
 3,  6, 0.43277
 3,  1, 0.48786
 9,  0, 0.52763
 0,  2, 0.56920
 4,  3, 0.59115
 2,  7, 0.65159
We found 9 edges from node 1 to node 8
 1,  0
 0,  2
 2,  7
 0,  5
 0,  9
 1,  3
 3,  4
 3,  6
 6,  8


In [83]:
# this is the results you should get:
'''
The minimum spanning tree:
 8,  6, 0.00920
 0,  5, 0.02232
 1,  0, 0.25507
 6,  3, 0.43277
 1,  3, 0.48786
 0,  9, 0.52763
 2,  0, 0.56920
 3,  4, 0.59115
 7,  2, 0.65159
We found 3 edges from node 1 to node 8
 1,  3
 3,  6
 6,  8
'''
print()

### Q 1.1 (A)
The minimum spanning tree does not change when each edge weight is incremented by 1
### Q 1.2 (B)
Removing the root node is the only way to disconnect the tree

### Q 1.3 (C)
?

### Q 1.4 (D)
6 minutes 

### Q 1.5 (B)
{him, ham, cat, bat}

### Q 1.6 (D)
bacca






# Q 11
Given an L table, the length of the LCS is given by the value at the bottom right of the table. In this case, the value at the bottom right is 6. Therefore the length of the LCS is 6.

# Q7

In [105]:
def calc(n): 
    k=0 #O(1)
    i = n/2 #O(1)
    while(i<=n): #since i is incrementing by 1 each time, this has to run a maximum (worst case) of n/2 times as it starts from n/2
        i=i+1 #O(1)
        j=2 #O(1)
        #since j is incrementing by a factor of 2 each time, the remaining interval decreases by a factor of 2. 
        # Thus this has worse case time complexity of O(logn)
        while(j<=n):
            j=j*2 #O(1)
            k = k + n/2 #O(1)
    return k,counter

Therefore, 
* The time complexity of the above function is $O(1) + O(1) + O(n/2 \times logn \times 1 \times 1) \approx O(nlogn)$
* The value of k :  
    A value of $n/2$ gets added to k (starting from 0), $nlogn$ times. Therefore, $k \approx O(n^2logn)$  

# Q6


Pseudocode - 

Given N x N board of characters, from a randomly chosen grid point, there are 8 possible directions to traverse.  
(1)Up, (2)Down, (3)Left, (4)Right, (5)Top right, (6)Top left, (7)Bottom right, (8)Bottom left.  

```
For row in range(N) # O(N)
    For column in range(N)  # O(N)
        grid point index = (row,col) 
        For each word in the given set of N words, # O(N)
            - Traverse in all 8 directions on the board of characters from the grid point, 
                for a total distance (in each direction) equalling the length of the word to be found.   
            - Ensure the program(crawler) remains within the bounds of the board. 
            - Use brute force technique to make character comparisons 
            - Worst case number of character comparisons to be made, per word, per grid point, 
                accoutning for all 8 directions is 8 x len(word)
``` 

Therefore, worst case time complexity is $O(N \times N \times N \times 8 \times len(longest\ word)) \approx O(N^3)$, assuming $len(longest\ word) << N,\ and\ N >> 8$
    